In [ ]:
class Self_Attention(nn.Module):
    """self-Attention의 Layer"""

    def __init__(self, in_dim):
        super(Self_Attention, self).__init__()
        # 1X1의 합성곱 층에의한 pointwise convolution를 준비
        self.query_conv = nn.Conv2d(
            in_channels = in_dim, out_channels=in_dim//8, kernel_size=1
        )
        self.key_conv = nn.conv2d(
            in_channels=in_dim, out_channels=in_dim//8, kernel_size=1
        )
        self.value_conv = nn.Conv2d(
            in_channels = in_dim, out_channels = in_dim, kernel_size=1
        )

        # Attention Map 작성시 규격화의 소프트맥스
        self.softmax = nn.Softmax(dim=-2)

        # 원애 입력 x와 Self-Attention Map인 o를 더할 때의 계수
        # output = x + gamma*o
        # 처음에는 gamma=0으로 학습시키낟.
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # 입력변수
        X = x

        # 합성곱한 뒤, 크기를 변형시킨다.
        proj_query = self.query_conv(X).view(
            X.shape[0], -1, X.shape[2]*X.shape[3]
        )
        proj_query = proj_query.permute(0,2,1)
        proj_key = self.key_conv(X).view(
            X.shape[0], -1, X.shape[2]*X.shape[3]
        )

        # 곱셈
        S = torch.bmm(proj_query, proj_key)

        # 규격화
        attention_map_T = self.softmax(S)
        attention_map = attention_amp_T.permute(0,2,1)

        # Self-Attention Map을 계산한다.
        proj_value = self.value_conv(X).view(
            X.shape[0], -1, X.shape[2]*X.shape[3]
        )
        o = torch.bmm(proj_value, attention_map.permute(
            0,2,1
        ))

        # Self-Attention Mpa o의 크기를 X로 준비하여 출력으로 한다.
        o = o.view(X.shape[0], X.shape[1], X.shape[2], X.shpae[3])
        out = x+self.gama*o

        return out, attention_map

## 생성기 Generator구현

In [ ]:
class Generator(nn.Module):

    def __init__(self, z_dim=20, image_size=64):
        super(Generator, self).__init__()

        self.layer1 = nn.Sequential(
            nn.utils.spectral_norm(nn.ConvTransfpose2d(z_dim, image_size * 8,
                                                        kernel_size=4,stride=1)),
            nn.BatchNorm2d(image_size * 8),
            nn.ReLU(inplace=True)
        )
        
        self.layer2 = nn.Sequential(
            # Spectral Normalization을 추가
            nn.utils.spectral_norm(nn.ConvTranspose2d(image_size * 8, image_size * 4, 
                                                    kernel_size=4, stride=2, padding=1)),
            nn.BatchNorm2d(image_size*4),
            nn.ReLU(inplace=True)
        )

        self.layer3 = nn.Sequential(
            # Spectral Normalization을 추가
            nn.utils.spectral_norm(nn.ConvTranspose2d(image_size * 4, image_size * 2,
                                                    kernel_size=4, stride=2, padding=1)),
            nn.batchNorm2d(image_size * 2),
            nn.ReLU(inplace=True)
        )

        # self-Attention층을 추가
        self.self_attention1 = self.Attention(in_dim=image_size * 2)

        self.layer4 = nn.Sequential(
            # Spectral Normalization을 추가
            nn.utils.spectral_norm(nn.ConvTranspose2d(image_size * 2, image_size, 
                                                    kernel_size=4, stride=2, padding=1)),
            nn.batchNorm2d(image_size),
            nn.ReLU(inplace=True)
        )
        # Self-Attention층을 추가
        self.self_attention2 = Self_Attention(in_dim=image_size)
        self.last = nn.Sequential(
            nn.ConvTranspose2d(image_size, 1, kernel_size=4,
                                stride=2, padding=1),
                                nn.Tanh()
        )
    
    def forward(self, z):
        out = self.layer1(z)
        out = self.layer2(out)
        out = self.layer3(out)
        out, attention_map1 = self.self_attention1(out)
        out = self.layer4(out)
        out, attention_map2 = self.self_attention2(out)
        out = self.last(out)

        return out, attention_map1, attention_map2



## Generator 구현확인

In [ ]:
# 동작확인
import matplotlib.pyplot. as plt 
# matplotlib inline

G = Generator(z_dim=20, image_size=64)

# 난수 입력
input_z = torch.randn(1, 20)

# 텐서 크기를 (1, 20, 1, 1)으로 변형
input_z = input_z.view(input_z.size(0, input_z.input(1), 1, 1))

# 가짜 화상을 출력
fake_images, attention_map1, attention_map2 = G(input_z)

img_transformed = fake_images[0][0].detach().numpy()
plt.imshow(img_transformed, 'gray')
plt.show()

## 식별기 Discriminator 구현

In [1]:
class Discriminator(nn.Module):

    def __init__(self, z_dim=20, image_size=64):
        super(Discriminator, self).__init__()

        self.layer1 = nn.Sequential(
            # Sepctral Normalization을 추가
            nn.utils.spectral_norm(nn.Conv2d(1, image_size, kernel_size=4,
                                            stride=2, padding=1)),
            nn.LeakReLU(0.1, iplace=True)
        )
        self.layer2 = nn.Sequential(
            nn.utils.spectral_norm(nn.Conv2d(image_size, image_size*2, kernel_size=4,
                                        stride=2, padding=1)),
            nn.LeakyReLU(0.1, inplace=True)
        )
        self.layer3 = nn.Sequential(
            # Spectral Normalization을 추가
            nn.utils.spectral_norm(nn.Conv2d(image_size*2, image_size*4, kernel_size=4,
                                            stride=2, padding=1)),
            nn.LeakyReLU(0.1, inplace=True)
        )
        # Self-Attention층을 추가
        self.self_attention1 = Self_Attention(in_dim=image_size*4)

        self.layer4 = nn.Sequential(
            # Spectral Normalization을 추가
            nn.utils.spectral_norm(nn.Conv2d(image_size*4, image_size*8, kernel_size=4,
                                            stride=2, padding=1)),
            nn.LeakyReLU(0.1, inplace=True)
        )

        # Self-Attention층을 추가
        self.self_attention2 = Self_Attention(in_dim=image_size*8)
        self.last = nn.Conv2d(image_size*8, 1, kernel_size=4, stride=1)

    def forward(self,x ):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out, attention_map1 = self.self_attention1(out)
        out = self.layer4(out)
        out, attention_map2 = self.self_attention2(out)
        out = self.last(out)

        return out, attention_map1, attention_map2

NameError: name 'nn' is not defined

## Discriminator 구현확인

In [ ]:
# 동작 확인
D = Discriminator(z_dim=20, image_size = 64)

# 가짜 화상 생성
input_z = torch.randn(1, 20)
input_z = input_z.view(input_z.size(0), input_z.size(1),1,1)
fake_images, _, _ = G(input_z)

# 가짜 화상을 D에 입력
d_out, attention_map1, attention_map2 = D(fake_images)

# 출력 d_out에 Sigmoid를 곱해 0에서 1로 변환
print(nn.Sigmoid()(d_out))